# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import logging
import azureml
import os

from azureml.core import Environment
from azureml.core import ScriptRunConfig
from azureml.core.runconfig import DockerConfiguration
from azureml.core.dataset import Dataset


from azureml.core import Workspace, Experiment

from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

from azureml.core.runconfig import EnvironmentDefinition, CondaDependencies

#from azureml.train.estimator import Estimator
from azureml.train.hyperdrive import (
    BayesianParameterSampling,
    HyperDriveConfig,
    quniform,
    uniform,
    choice,
    PrimaryMetricGoal,
)

from azureml.widgets import RunDetails

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.41.0


## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()
experiment_name = 'oj-forecast-hyperdrive'

experiment=Experiment(ws, experiment_name)
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = experiment.start_logging()

Workspace name: quick-starts-ws-198405
Azure region: southcentralus
Subscription id: 976ee174-3882-4721-b90a-b5fef6b72f24
Resource group: aml-quickstarts-198405


In [3]:
# Create or attach a compute cluster

cluster_name = "canina-cluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('This compute target already exists.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size="Standard_D2_V2",
                                                          max_nodes=4)
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)
    print('A new compute target has been created.')
    
compute_target.wait_for_completion(show_output=True, min_node_count=0, timeout_in_minutes=10)

This compute target already exists.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [4]:
train_dataset = Dataset.get_by_name(ws, name='train')
train_id = train_dataset.id

In [6]:
auxi_dataset = Dataset.get_by_name(ws, name='auxi')
auxi_id = auxi_dataset.id

# skip

In [ ]:
DATA_DIR = os.path.join(os.path.abspath(os.getcwd()), "ojdata")
path_on_datastore = "data"

In [ ]:
ws.get_default_datastore()

In [ ]:
# Upload data to datastore

ds = ws.get_default_datastore()
ds.upload(
    src_dir=DATA_DIR,
    target_path=path_on_datastore,
    overwrite=True,
    show_progress=False)

In [ ]:
# Get data reference object for the data path
ds_data = ds.path(path_on_datastore)
print(ds_data)

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [7]:
# Configure Docker environment 
env = EnvironmentDefinition()
env.python.user_managed_dependencies = False
env.python.conda_dependencies = CondaDependencies.create(
    conda_packages=["pandas", "numpy", "scipy", "scikit-learn", "lightgbm", "joblib"],
    python_version="3.6.2"
)
env.python.conda_dependencies.add_channel("conda-forge")

docker_config = DockerConfiguration(use_docker=True)

In [8]:
# Training script
script_folder = "./hyperdrive"
train_script_name = "train.py"

src = ScriptRunConfig(source_directory=script_folder,
                      script=train_script_name,
                      arguments=["--train", train_id, "--auxi", auxi_id],
                      compute_target=compute_target,
                      environment=env,
                      docker_runtime_config=docker_config)

# Create the different params used during training
param_sampling = BayesianParameterSampling(
    {
        "--num-leaves": quniform(8, 128, 1),
        "--min-data-in-leaf": quniform(20, 500, 10),
        "--learning-rate": choice(
            1e-4, 1e-3, 5e-3, 1e-2, 1.5e-2, 2e-2, 3e-2, 5e-2, 1e-1
        ),
        "--feature-fraction": uniform(0.2, 1),
        "--bagging-fraction": uniform(0.1, 1),
        "--bagging-freq": quniform(1, 20, 1),
        "--max-rounds": quniform(50, 2000, 10),
        "--max-lag": quniform(3, 40, 1),
        "--window-size": quniform(3, 40, 1)
    }
)

# Create an estimator and a hyperdrive config
#script_params = {"--data-folder": ds_data.as_mount()}

"""estimator = Estimator(
    source_directory=script_folder,
    script_params=script_params,
    compute_target=compute_target,
    use_docker=True,
    entry_script=train_script_name,
    environment_definition=env
)"""

hyperdrive_run_config = HyperDriveConfig(
    run_config=src,
    hyperparameter_sampling=param_sampling,
    primary_metric_name="MAPE",
    primary_metric_goal=PrimaryMetricGoal.MINIMIZE,
    max_total_runs=25,
    max_concurrent_runs=4
)

For best results with Bayesian Sampling we recommend using a maximum number of runs greater than or equal to 20 times the number of hyperparameters being tuned. Recommendend value:180.


In [9]:
# Submit the experiment
hyperdrive_run = experiment.submit(config=hyperdrive_run_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [10]:
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [11]:
# We need to wait before retrieving the best model
hyperdrive_run.wait_for_completion(show_output=True)

RunId: HD_44c3244f-c7b1-43fa-9424-3aa531fb1c42
Web View: https://ml.azure.com/runs/HD_44c3244f-c7b1-43fa-9424-3aa531fb1c42?wsid=/subscriptions/976ee174-3882-4721-b90a-b5fef6b72f24/resourcegroups/aml-quickstarts-198405/workspaces/quick-starts-ws-198405&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Execution Summary
RunId: HD_44c3244f-c7b1-43fa-9424-3aa531fb1c42
Web View: https://ml.azure.com/runs/HD_44c3244f-c7b1-43fa-9424-3aa531fb1c42?wsid=/subscriptions/976ee174-3882-4721-b90a-b5fef6b72f24/resourcegroups/aml-quickstarts-198405/workspaces/quick-starts-ws-198405&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254



{'runId': 'HD_44c3244f-c7b1-43fa-9424-3aa531fb1c42',
 'target': 'canina-cluster',
 'status': 'Completed',
 'startTimeUtc': '2022-06-14T06:13:07.448823Z',
 'endTimeUtc': '2022-06-14T06:36:25.292328Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name": "MAPE", "goal": "minimize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '4555839a-8f0a-4a4b-9d7d-4b0129b3d31e',
  'user_agent': 'python/3.8.5 (Linux-5.4.0-1077-azure-x86_64-with-glibc2.10) msrest/0.6.21 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.41.0',
  'space_size': 'infinite_space_size',
  'score': '31.42248373578293',
  'best_child_run_id': 'HD_44c3244f-c7b1-43fa-9424-3aa531fb1c42_5',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_44c3244f-c7b1-43fa-9424-3aa531fb1c42_5'},
 'inputDatasets': [],
 'outputDatasets': [],
 'logFiles': {'azureml-logs/hyperdrive.txt': 'https://mlstrg198405.blob.core.win

In [12]:
# View of the different models trained and their performance
hyperdrive_run.get_metrics()

{'HD_44c3244f-c7b1-43fa-9424-3aa531fb1c42_22': {'MAPE': 55.326612586646974},
 'HD_44c3244f-c7b1-43fa-9424-3aa531fb1c42_24': {'MAPE': 35.2596562416394},
 'HD_44c3244f-c7b1-43fa-9424-3aa531fb1c42_23': {'MAPE': 33.84939482793721},
 'HD_44c3244f-c7b1-43fa-9424-3aa531fb1c42_19': {'MAPE': 36.30777380577047},
 'HD_44c3244f-c7b1-43fa-9424-3aa531fb1c42_21': {'MAPE': 34.340766766344366},
 'HD_44c3244f-c7b1-43fa-9424-3aa531fb1c42_20': {'MAPE': 33.46150386008222},
 'HD_44c3244f-c7b1-43fa-9424-3aa531fb1c42_18': {'MAPE': 36.191214103840025},
 'HD_44c3244f-c7b1-43fa-9424-3aa531fb1c42_16': {'MAPE': 42.83248740112451},
 'HD_44c3244f-c7b1-43fa-9424-3aa531fb1c42_17': {'MAPE': 74.41475755133055},
 'HD_44c3244f-c7b1-43fa-9424-3aa531fb1c42_14': {'MAPE': 34.52913265649079},
 'HD_44c3244f-c7b1-43fa-9424-3aa531fb1c42_15': {'MAPE': 34.514489359889424},
 'HD_44c3244f-c7b1-43fa-9424-3aa531fb1c42_13': {'MAPE': 33.9780542530078},
 'HD_44c3244f-c7b1-43fa-9424-3aa531fb1c42_11': {'MAPE': 35.06486859917399},
 'HD_44c32

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [13]:
best_run_hyperdrive = hyperdrive_run.get_best_run_by_primary_metric()
# Print the best model's Run Id and metrics

best_metrics = best_run_hyperdrive.get_metrics()
print('Run Id : ', best_run_hyperdrive.id)
print('Metrics :',best_metrics)

Run Id :  HD_44c3244f-c7b1-43fa-9424-3aa531fb1c42_5
Metrics : {'MAPE': 31.42248373578293}


In [23]:
best_run_hyperdrive.get_details()['runDefinition']['arguments']

['--train',
 'ef9d4108-b6f5-4ebc-aa80-0595f5c4f062',
 '--auxi',
 'bf15b189-3137-49fd-9282-65341a56f0de',
 '--num-leaves',
 '128',
 '--min-data-in-leaf',
 '100',
 '--learning-rate',
 '0.05',
 '--feature-fraction',
 '0.4974947375018502',
 '--bagging-fraction',
 '0.4078898883596621',
 '--bagging-freq',
 '10',
 '--max-rounds',
 '1510',
 '--max-lag',
 '36',
 '--window-size',
 '33']

In [24]:
best_run_hyperdrive.get_file_names()

['logs/azureml/dataprep/0/backgroundProcess.log',
 'logs/azureml/dataprep/0/backgroundProcess_Telemetry.log',
 'logs/azureml/dataprep/0/rslex.log',
 'logs/azureml/dataprep/0/rslex.log.2022-06-14-06',
 'system_logs/cs_capability/cs-capability.log',
 'system_logs/hosttools_capability/hosttools-capability.log',
 'system_logs/lifecycler/execution-wrapper.log',
 'system_logs/lifecycler/lifecycler.log',
 'system_logs/lifecycler/vm-bootstrapper.log',
 'system_logs/metrics_capability/metrics-capability.log',
 'user_logs/std_log.txt']

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
# Register the best model
best_hyperdrive_model = best_run_hyperdrive.register_model(model_name = 'hyperdrive-lgb', model_path = './outputs/model.joblib')

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

